In [1]:
import numpy as np
from timeit import default_timer as timer
from matplotlib import pyplot
import math
import numbapro # We use check_cuda() and vectorize
from numbapro import vectorize, autojit, jit, cuda #vectorize for functions and cuda for cuda-u functions
numbapro.check_cuda()

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
numbapro:1: ImportWarning: The numbapro package is deprecated in favour of the accelerate package. Please update your code to use equivalent functions from accelerate.


------------------------------libraries detection-------------------------------
Finding cublas
	located at /home/john/anaconda2/lib/libcublas.so.7.0.28
	trying to open library...	ok
Finding cusparse
	located at /home/john/anaconda2/lib/libcusparse.so.7.0.28
	trying to open library...	ok
Finding cufft
	located at /home/john/anaconda2/lib/libcufft.so.7.0.35
	trying to open library...	ok
Finding curand
	located at /home/john/anaconda2/lib/libcurand.so.7.0.28
	trying to open library...	ok
Finding nvvm
	located at /home/john/anaconda2/lib/libnvvm.so.3.0.0
	trying to open library...	ok
	finding libdevice for compute_20...	ok
	finding libdevice for compute_30...	ok
	finding libdevice for compute_35...	ok
-------------------------------hardware detection-------------------------------
Found 1 CUDA devices
id 0        GeForce GTX 870M                              [SUPPORTED]
                      compute capability: 3.0
                           pci device id: 0
                              

True

In [2]:
import numba.cuda  
#this line from nbviewer.ipython.org/github/ContinuumIO/numbapro-examples/blob/master/webinars/2014_06_17/intro_to_gpu_python.ipynb
my_gpu = numba.cuda.get_current_device()
print "Running on GPU:", my_gpu.name
cores_per_capability = {
    1: 8,
    2: 32,
    3: 192,
}
cc = my_gpu.compute_capability
print "Compute capability: ", "%d.%d" % cc, "(Numba requires >= 2.0)"
majorcc = cc[0]
print "Number of streaming multiprocessor:", my_gpu.MULTIPROCESSOR_COUNT
cores_per_multiprocessor = cores_per_capability[majorcc]
print "Number of cores per mutliprocessor:", cores_per_multiprocessor
total_cores = cores_per_multiprocessor * my_gpu.MULTIPROCESSOR_COUNT
print "Number of cores on GPU:", total_cores

Running on GPU: GeForce GTX 870M
Compute capability:  3.0 (Numba requires >= 2.0)
Number of streaming multiprocessor: 7
Number of cores per mutliprocessor: 192
Number of cores on GPU: 1344


In [3]:
def iter_loadtxt(filename, delimiter=',', skiprows=1, dtype=float):
    def iter_func():
        with open(filename, 'r') as infile:
            for _ in range(skiprows):
                next(infile)
            for line in infile:
                line = line.rstrip().split(delimiter)
                for item in line:
                    yield dtype(item)
        iter_loadtxt.rowlength = len(line)

    data = np.fromiter(iter_func(), dtype=dtype)
    data = data.reshape((-1, iter_loadtxt.rowlength))
    return data

In [ ]:
file_load = iter_loadtxt('/media/john/My Passport/Sphere project data/run2--curl/data4.csv',delimiter=',',skiprows=1,dtype=float)
    
polar0 = np.asarray([row[1] for row in file_load])
polar1 = np.asarray([row[2] for row in file_load])
polar2 = np.asarray([row[3] for row in file_load])
     
polar0ave = np.mean(polar0)
polar1ave = np.mean(polar1)
polar2ave = np.mean(polar2)
    
Polar0fluct = np.asarray(polar0 - polar0ave)
Polar1fluct = np.asarray(polar1 - polar1ave)
Polar2fluct = np.asarray(polar2 - polar2ave)
    
r0 = np.asarray([row[4] for row in file_load])
r1 = np.asarray([row[5] for row in file_load])
r2 = np.asarray([row[6] for row in file_load])